In [14]:
pip install deepmultilingualpunctuation

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import os
import re
import pandas as pd
import nltk
import ssl
import torch
import spacy
import numpy as np
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from deepmultilingualpunctuation import PunctuationModel
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from scipy.linalg import triu

In [17]:
modelpunct = PunctuationModel()
# Load pre-trained BERT model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:168: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="none"` instead.
  warnings.warn(


In [18]:
# Load SpaCy model for NLP tasks
nlp = spacy.load('en_core_web_sm')

# Function to group consecutive words into sentences
def group_words_into_sentences(df):
    sentences = ''
    previous_word = None

    for _, row in df.iterrows():
        word = row['word']
        if word != previous_word:
            sentences += " " +word
        previous_word = word

    return sentences.strip()

# Function to chunk text for the punctuation pipeline
def chunk_text(text, chunk_size=512):
    words = text.split()
    for i in range(0, len(words), chunk_size):
        yield ' '.join(words[i:i + chunk_size])

# Function to punctuate chunks
def punctuate_chunks(chunks):
    punctuated_chunks = []
    for chunk in chunks:
        punctuated_text = modelpunct.restore_punctuation(chunk)
        punctuated_chunks.append(punctuated_text)
    return punctuated_chunks

def combine_chunks(chunks):
    return ' '.join(chunks)

# Function to extract NLP features using SpaCy
def extract_nlp_features(text):
    doc = nlp(text)
    features = []
    for token in doc:
        features.append({
            'word': token.text,
            'lemma': token.lemma_,
            'tag': token.tag_,
            'dep': token.dep_,
        })
    return features


In [29]:
# Function to remove records with transcripts '000' or '999'
def filter_special_cases(df):
    return df[~df['word'].isin(['000', '999'])]

# Function to remove column that are always null
def filter_null(df):
    columns_to_drop = df.columns[(df == 0).all()]
    # Drop the identified columns
    return df.drop(columns=columns_to_drop)

# Function to remove records with filler words
def filter_fillers(df):
    # Define a list of filler words
    filler_words = ['like', 'god', 'oh', 'jesus']
    return df[~df['word'].isin(filler_words)]

# Function to retain the last 300 joint positions of words spanning more than 300 frames (longer than ~2 seconds of speech)
def retain_last_n_jp(df, max_rows):
    previous_word = None
    drop = []
    rows = []
    # Loop through each row
    for _,row in df.iterrows():
        current_word = row['word']
        frame = row['frame']
        if current_word != previous_word and previous_word!= None:
            if len(rows) > max_rows:
                drop = df[df['frame'].isin(rows[:-max_rows])].index
                df.drop(drop, axis =0, inplace = True)
                rows = []
        else:
            rows.append(frame)

        previous_word = current_word
    return df

# Handling Missing Values
def handle_missing_values(df):
    df.fillna(method='ffill', inplace=True)

    nlp_features = ['word', 'lemma', 'tag', 'dep']

    for feature in nlp_features:
      df[feature].fillna('', inplace=True)

    return df

# Normalization/Standardization
def normalize_features(df, numerical_features):
    scaler = StandardScaler()
    df[numerical_features] = scaler.fit_transform(df[numerical_features])
    return df

# Feature Engineering (Velocities and Accelerations)
def compute_rotation_dynamics(df):
    df['time_diff'] = df['time'].diff().fillna(0)
    joints = ['RightShoulder', 'RightArm', 'RightForeArm', 'RightHand', 'LeftShoulder', 'LeftArm', 'LeftForeArm', 'LeftHand']

    dynamics = pd.DataFrame(index=df.index)
    for joint in joints:
        for axis in ['x', 'y', 'z']:
            rotation_col = f'{joint}_rotation_{axis}'
            dynamics[f'{joint}_angular_velocity_{axis}'] = df[rotation_col].diff().fillna(0) / df['time_diff']
            dynamics[f'{joint}_angular_acceleration_{axis}'] = dynamics[f'{joint}_angular_velocity_{axis}'].diff().fillna(0) / df['time_diff']

    df.drop(columns=['time_diff'], inplace=True)
    df = pd.concat([df, dynamics], axis=1)
    return df


# Function to compute relative positions and angles based on parent-child relationships
def compute_relative_rotations(df):
    parent_map = {
        'Hips': None,
        'Spine': 'Hips',
        'Spine1': 'Spine',
        'Spine2': 'Spine1',
        'Spine3': 'Spine2',
        'RightShoulder': 'Spine3',
        'RightArm': 'RightShoulder',
        'RightForeArm': 'RightArm',
        'RightHand': 'RightForeArm',
        'LeftShoulder': 'Spine3',
        'LeftArm': 'LeftShoulder',
        'LeftForeArm': 'LeftArm',
        'LeftHand': 'LeftForeArm'
    }
    for child, parent in parent_map.items():
        if parent is not None:
            for axis in ['x', 'y', 'z']:
                # Compute relative rotations
                df[f'{child}_rel_rotation_{axis}'] = df[f'{child}_rotation_{axis}'] - df[f'{parent}_rotation_{axis}']
    return df

def extract_file_numbers(file_list):
    file_numbers = [int(re.findall(r'\d+', os.path.basename(file))[0]) for file in file_list]
    return file_numbers

In [22]:
def split_file_numbers(file_numbers, test_size=0.2, val_size=0.25, random_state=42):
    train_files, test_files = train_test_split(file_numbers, test_size=test_size, random_state=random_state)
    train_files, val_files = train_test_split(train_files, test_size=val_size, random_state=random_state)
    return train_files, val_files, test_files

# # Function to train an embedding model on the words
# def train_embedding_model(df, vector_size=100):
#   sentences = df['word'].values.reshape(-1, 1).tolist()
#   word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
#   return word2vec_model


# Function to get Word2Vec embeddings
def get_word2vec_embedding(word, model, vector_size):
    try:
        return model.wv[word].tolist()
    except KeyError:
        return [0] * vector_size

In [27]:
# Function to preprocess each CSV file
def preprocess_csv_file(filepath, output_path):

    df = pd.read_csv(filepath)
    # Apply preprocessing from the analysis of the data
    df['Hips_parent'].fillna('None', inplace=True)
    df['word'] = df['word'].astype(str)
    df_cleaned = filter_special_cases(df)
    df_cleaned = filter_null(df_cleaned)
    df_cleaned = filter_fillers(df_cleaned)
    df_cleaned = retain_last_n_jp(df_cleaned, max_rows=300)

    # Apply text preprocessing
    text = group_words_into_sentences(df_cleaned)
    chunks = list(chunk_text(text, chunk_size=100))
    punctuated_chunks = punctuate_chunks(chunks)
    punctuated_text = combine_chunks(punctuated_chunks)
    nlp = extract_nlp_features(punctuated_text)
    nlp_df = pd.DataFrame(nlp)


    # Initialize columns for NLP features in the enhanced DataFrame
    for feature in ['lemma', 'tag', 'dep']:
        df_cleaned[feature] = None

    # Merge NLP features with the cleaned DataFrame
    nlp_index = 0
    nlp_row_count = len(nlp_df)
    for index, row in df_cleaned.iterrows():
        word = row['word']
        if nlp_index < nlp_row_count and word == nlp_df.iloc[nlp_index]['word']:
            for feature in ['lemma', 'tag', 'dep']:
                df_cleaned.at[index, feature] = nlp_df.iloc[nlp_index][feature]
        else:
            if nlp_index < nlp_row_count - 1:
                nlp_index += 1
                if word == nlp_df.iloc[nlp_index]['word']:
                    for feature in ['lemma', 'tag', 'dep']:
                        df_cleaned.at[index, feature] = nlp_df.iloc[nlp_index][feature]


    df = handle_missing_values(df_cleaned)
    df = compute_rotation_dynamics(df)
    df = compute_relative_rotations(df)

    print(df.head)

    # Drop parent columns
    parent_columns = [col for col in df.columns if '_parent' in col]
    df.drop(columns=parent_columns, inplace=True)

    # Get embeddings for the transcript words
    sentences = df['word'].values.reshape(-1, 1).tolist()
    word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
    df['word_embedding'] = df['word'].apply(lambda x: get_word2vec_embedding(x, word2vec_model, 100))
    word_embeddings_df = pd.DataFrame(df['word_embedding'].to_list(), index=df.index)
    word_embeddings_df.columns = [f'word_emb_{i}' for i in range(100)]
    df = pd.concat([df, word_embeddings_df], axis=1)
    df.drop(columns=['word_embedding', 'word'], inplace=True)

    # One-hot encode NLP features and joint parent data
    nlp_features = ['lemma', 'tag', 'dep']
    df = pd.get_dummies(df, columns=nlp_features)

    # Save final data
    df.to_csv(output_path, index=False)
    print(f"preprocessed features and saved data for {filepath} to {output_path}")



In [24]:
# Process each CSV file
def process_files (file_set, output_dir):
    for file_number in file_set:
        file_path = os.path.join(csv_dir, f'features_{file_number}.csv')
        output_path = os.path.join(output_dir, f'preprocessed_features_{file_number}.csv')
        preprocess_csv_file(file_path, output_path)


In [ ]:
# Process each CSV file
def process_files_train (file_set, output_dir):
    for file_number in file_set:
      if file_number!=11 or file_number!=25 or file_number!=18:
          print(file_number)
          file_path = os.path.join(csv_dir, f'features_{file_number}.csv')
          output_path = os.path.join(output_dir, f'preprocessed_features_{file_number}.csv')
          preprocess_csv_file(file_path, output_path)

In [25]:
# Directory containing the feature CSV files
csv_dir = '/content/drive/MyDrive/Unipi_Thesis/ExtractedFeatures'

# List all feature CSV files in the directory
csv_files = [f for f in os.listdir(csv_dir) if f.startswith('features_') and f.endswith('.csv')]


In [30]:
# Splitting train-val-test files
file_numbers = extract_file_numbers(csv_files)
train_files, val_files, test_files = split_file_numbers(file_numbers)
print("train", train_files)
print("test", test_files)
print("tval", val_files)

# Directories to save the processed datasets
train_output_dir = '/content/drive/MyDrive/Unipi_Thesis/Sets/Train'
val_output_dir = '/content/drive/MyDrive/Unipi_Thesis/Sets/Validation'
test_output_dir = '/content/drive/MyDrive/Unipi_Thesis/Sets/Test'

# Ensure the directories exist
os.makedirs(train_output_dir, exist_ok=True)
os.makedirs(val_output_dir, exist_ok=True)
os.makedirs(test_output_dir, exist_ok=True)

# Process and save training, validation, and test files
process_files(train_files, train_output_dir)
process_files(val_files, val_output_dir)
process_files(test_files, test_output_dir)

print("Preprocessing complete and datasets saved.")

train [11, 25, 18, 27, 17, 23, 10, 4, 30, 5, 21, 14, 26]
test [19, 13, 1, 12, 22]
tval [16, 2, 7, 15, 8]


<ipython-input-29-d2e2d969e10c>:92: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{child}_rel_rotation_{axis}'] = df[f'{child}_rotation_{axis}'] - df[f'{parent}_rotation_{axis}']
<ipython-input-29-d2e2d969e10c>:92: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{child}_rel_rotation_{axis}'] = df[f'{child}_rotation_{axis}'] - df[f'{parent}_rotation_{axis}']
<ipython-input-29-d2e2d969e10c>:92: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has po

<bound method NDFrame.head of        take_id  frame        time    word  Hips_position_x  Hips_position_y  \
195         11    196    4.813333      so         -20.8610          91.5823   
231         11    232    5.113333      so         -15.2218          91.8221   
232         11    233    5.121666      so         -15.0729          91.8261   
233         11    234    5.129999      so         -14.9166          91.8272   
234         11    235    5.138333      so         -14.7562          91.8271   
...        ...    ...         ...     ...              ...              ...   
75818       11  78031  653.438073  people         -11.8670          90.8276   
75819       11  78032  653.446407  people         -11.5817          90.7976   
75820       11  78033  653.454740  people         -11.2977          90.7615   
75821       11  78034  653.463073  people         -11.0334          90.7318   
75822       11  78035  653.471407  people         -10.7572          90.6946   

       Hips_position_

<ipython-input-29-d2e2d969e10c>:92: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{child}_rel_rotation_{axis}'] = df[f'{child}_rotation_{axis}'] - df[f'{parent}_rotation_{axis}']
<ipython-input-29-d2e2d969e10c>:92: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{child}_rel_rotation_{axis}'] = df[f'{child}_rotation_{axis}'] - df[f'{parent}_rotation_{axis}']
<ipython-input-29-d2e2d969e10c>:92: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has po

<bound method NDFrame.head of        take_id  frame        time   word  Hips_position_x  Hips_position_y  \
8           25      8   11.746667  right        -0.185901          89.0822   
9           25      9   11.755000  right        -0.122105          89.1139   
10          25     10   11.763333  right        -0.061020          89.1478   
11          25     11   11.771667  right         0.000764          89.1839   
12          25     12   11.780000  right         0.063035          89.2216   
...        ...    ...         ...    ...              ...              ...   
75997       25  78223  663.538073   yeah       -10.367000          90.4133   
75998       25  78224  663.546406   yeah       -10.426300          90.4027   
75999       25  78225  663.554739   yeah       -10.489000          90.3882   
76000       25  78226  663.563073   yeah       -10.555900          90.3687   
76001       25  78227  663.571406   yeah       -10.625700          90.3513   

       Hips_position_z  Hips_rota

...

<bound method NDFrame.head of        take_id  frame        time  word  Hips_position_x  Hips_position_y  \
319          1    319    7.468332   the          7.94736          92.7058   
320          1    320    7.476666   the          7.96694          92.7059   
321          1    321    7.484999   the          7.98878          92.7046   
322          1    322    7.493332   the          8.00875          92.7042   
323          1    323    7.501666   the          8.02555          92.7020   
...        ...    ...         ...   ...              ...              ...   
71110        1  73137  614.284756  okay        -12.08830          91.1470   
71111        1  73138  614.293090  okay        -12.28170          91.0945   
71112        1  73139  614.301423  okay        -12.48870          91.0381   
71113        1  73140  614.309756  okay        -12.68440          90.9658   
71114        1  73141  614.318090  okay        -12.86770          90.8776   

       Hips_position_z  Hips_rotation_x  Hips

<ipython-input-29-d2e2d969e10c>:92: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{child}_rel_rotation_{axis}'] = df[f'{child}_rotation_{axis}'] - df[f'{parent}_rotation_{axis}']
<ipython-input-29-d2e2d969e10c>:92: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{child}_rel_rotation_{axis}'] = df[f'{child}_rotation_{axis}'] - df[f'{parent}_rotation_{axis}']
<ipython-input-29-d2e2d969e10c>:92: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has po

<bound method NDFrame.head of        take_id  frame        time  word  Hips_position_x  Hips_position_y  \
398         12    399    6.124999    so        -25.96940          92.2268   
425         12    426    6.349999    so        -27.45790          91.8890   
426         12    427    6.358332    so        -27.56130          91.8739   
427         12    428    6.366665    so        -27.67230          91.8641   
428         12    429    6.374999    so        -27.79440          91.8559   
...        ...    ...         ...   ...              ...              ...   
75921       12  78144  653.999740  that         -6.42185          91.0093   
75922       12  78145  654.008073  that         -6.39394          90.9807   
75923       12  78146  654.016406  that         -6.37309          90.9483   
75924       12  78147  654.024740  that         -6.33872          90.9097   
75925       12  78148  654.033073  that         -6.30204          90.8678   

       Hips_position_z  Hips_rotation_x  Hips

<ipython-input-29-d2e2d969e10c>:92: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{child}_rel_rotation_{axis}'] = df[f'{child}_rotation_{axis}'] - df[f'{parent}_rotation_{axis}']
<ipython-input-29-d2e2d969e10c>:92: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{child}_rel_rotation_{axis}'] = df[f'{child}_rotation_{axis}'] - df[f'{parent}_rotation_{axis}']
<ipython-input-29-d2e2d969e10c>:92: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has po

<bound method NDFrame.head of        take_id  frame        time  word  Hips_position_x  Hips_position_y  \
224         22    224   18.166666  okay        -23.56390          90.0797   
225         22    225   18.174999  okay        -23.61090          90.0953   
226         22    226   18.183333  okay        -23.65910          90.1100   
227         22    227   18.191666  okay        -23.70400          90.1276   
228         22    228   18.199999  okay        -23.74830          90.1457   
...        ...    ...         ...   ...              ...              ...   
70536       22  72703  622.158091  skin         -5.71802          90.2729   
70537       22  72704  622.166424  skin         -5.54472          90.2291   
70538       22  72705  622.174758  skin         -5.36757          90.1873   
70539       22  72706  622.183091  skin         -5.18496          90.1483   
70540       22  72707  622.191424  skin         -4.99932          90.1145   

       Hips_position_z  Hips_rotation_x  Hips

In [31]:
import shutil

In [32]:
# Define the folder name you want to zip
folder_name = '/content/drive/MyDrive/Unipi_Thesis/Sets'
output_filename = '/content/drive/MyDrive/Unipi_Thesis/sets_zipped_features.zip'

# Zip the folder
shutil.make_archive(output_filename.replace('.zip', ''), 'zip', folder_name)

'/content/drive/MyDrive/Unipi_Thesis/sets_zipped_features.zip'

In [ ]:
    # # Add embeddings to the dataframe
    # vector_size = 100
    # for feature in nlp_features:
    #     df[f'{feature}_embedding'] = df[feature].apply(lambda x: get_embedding(x, embedding_models[feature], vector_size))


    # # Process embeddings in batches
    # batch_size = 32
    # vector_size = model.config.hidden_size

    # pca = PCA(n_components=50)  # Reduce to 50 dimensions
    # all_embeddings = []

    # # Add BERT embeddings to the dataframe for each NLP feature
    # for feature in nlp_features:
    #     embeddings = []
    #     for i in range(0, len(df), batch_size):
    #         batch_text = df[feature][i:i+batch_size].tolist()
    #         batch_embeddings = get_embeddings(batch_text)
    #         embeddings.extend(batch_embeddings)
    #     embeddings = pca.fit_transform(embeddings)  # Apply PCA
    #     embeddings_df = pd.DataFrame(embeddings, columns=[f'{feature}_emb_{i}' for i in range(pca.n_components_)])
    #     all_embeddings.append(embeddings_df)

    # # Combine all embeddings into the original dataframe
    # for i, feature in enumerate(nlp_features):
    #     df = pd.concat([df.reset_index(drop=True), all_embeddings[i].reset_index(drop=True)], axis=1)
    #     df.drop(columns=[feature], inplace=True)